### Importing libraries

In [5]:
import numpy as np
import pandas as pd
import sklearn
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import scipy
from datasets import load_dataset
from pydub import AudioSegment
import librosa

General Workflow:




1. Speech signal ---> Text




2. Text ----> Dirichlet Clusters




3. Dirichlet Cluster keywords ---> Search Engine / Youtube API







In [2]:
### Loading model to device

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [3]:
sample_audio_path = "C:/Users/yangz/OneDrive/Desktop/nus_masters/procrastinate/alejandro_voice.mp3"
# audio = scipy.io.wavfile.read(mono_audio_path)[1]
# print(audio)
# print(audio.dtype)

### We use the librosa package here to read mp3 files, for more information regarding the librosa package, you can visit: https://github.com/librosa/librosa

In [6]:
y, sr = librosa.load(sample_audio_path)
print(y)
print(sr)

[-1.0900003e-06 -1.8872288e-06  1.8608389e-06 ... -1.7513221e-06
  3.5237413e-06 -3.9590773e-06]
22050


In [7]:
output_result = pipe(y, generate_kwargs={"language": "english", "task": "transcribe"})
print(output_result["text"])

 Dum, ba-da-dum Ba-da-dum, dum, ba-da-dum Dum-ba-dum, dum, ba-da-dum Ba-da-dum, dum, ba-da-dum Dum-ba-da-dum Ba-da-dum, dum, ba-dum Ba-da-dum, dum, ba-da-dum Ba-da-dum, dum, ba-da-dum Woke up in the morning feeling kinda great Too many things I wanna do Walking down the streets I'll find a million ways To create something new Maybe I could ski across the Milky Way Paint a picture of your smile, oh I'll become a doctor and I'll save the day Or just sit for a while Roll or hide, this heart has more colors Than black and white, don't act so surprised Every day brings a different you You gotta have a canvas for dreams to come true People always have opinions on what you're doing And they'll try to put you down All you have to do is let go of what they're saying They don't know who's the best in town Wrong or right This heart has more colors than black and white Don't act so surprised, every day brings a different hue You gotta have a canvas for dreams to come true All the steps we take Bri

### Exporting the transcribed voice into a text file

In [8]:
output_file = "alejandro_voice_transcribed.txt"
transcribed_text = output_result["text"]
with open(output_file, "w") as dst:
    dst.write(transcribed_text)